## Predictive Modelling
  

In [1]:
#Initiating start time
import time
time_start = time.perf_counter()

In [2]:
from ipynb.fs.full.fuzz_model_dev import *

In [3]:
#Setting up test data

test_data.reset_index(drop=True, inplace=True)
y_test = test_data.loc[:,'heat']

In [4]:
#Fuzzifying test humidity

test_hum_val = test_data.iloc[:,0]
test_hum_df = fuzzify_hum(test_hum_val)

In [5]:
#Fuzzifying test temperature

test_temp_val = test_data.iloc[:,1]
test_temp_df = fuzzify_temp(test_temp_val)

In [6]:
#Defining the center of the fuzzy region

def fuzzr_center(a, lr, mr, hr):
    if a == 'low':
        return range_df.loc['min','heat']+lr
    elif a == 'mid':
        return med(range_df.loc['min','heat'], range_df.loc['max','heat'])+mr
    else:
        return range_df.loc['max','heat']+hr

In [7]:
# Applying a centroid defuzzification and evaluation

from sklearn.metrics import r2_score, mean_squared_error
ynum = np.zeros([y_test.shape[0], rule_fuzzy.shape[0]])
ydenom = np.zeros([y_test.shape[0], rule_fuzzy.shape[0]])

def defuzz(test_data, rule_fuzzy, lr, mr, hr):

    for i, fser in test_data.iterrows():
        for j, rser in rule_fuzzy.iterrows():
            col = rser.values
            rule_acx_num =test_hum_df.loc[i,col[0]] * test_temp_df.loc[i,col[1]]*fuzzr_center(col[2], lr, mr, hr)
            rule_acx_denom =test_hum_df.loc[i,col[0]]*test_temp_df.loc[i,col[1]]
            ynum[i,j] = rule_acx_num
            ydenom[i,j] = rule_acx_denom
        
    sum_ynum = np.sum(ynum, axis=1)
    sum_ydenom = np.sum(ydenom, axis=1)
    y_pred = sum_ynum/sum_ydenom


    model_r2 = r2_score(y_test, y_pred)
    model_mse = mean_squared_error(y_test, y_pred)
    
    print((lr, mr, hr), model_r2, model_mse)

In [8]:
print('parameter          r2_score             rmse')
defuzz(test_data, rule_fuzzy, -0.6, 1.0, 0)

parameter          r2_score             rmse
(-0.6, 1.0, 0) 0.9740261121057313 0.007093196396875854


In [9]:
time_elapsed = (time.perf_counter() - time_start)
print("%5.1f sec " % (time_elapsed))

 22.0 sec 
